In [ ]:
import serial
import threading
import time

# port = 'COM14'
port = '/dev/ttyUSB0'
baud_rate = 115200

escape_byte_bs = b'\xfa' # byte string of 0xfa
escape_byte = escape_byte_bs[0] # int type == 0xfa == 250

# for uart protocal,  -------------------- TB Tested --------------------------
def uart_decoder(data):
    # decode escape bytes
    result = b''

    i = 0
    while i < len(data):
        byte = data[i] # python gets the int type of single byte

        if byte != escape_byte:
            result += byte.to_bytes(1, 'big') # get the bytestring of this byte
            i += 1 # processed 1 byte for normal byte
            continue

        # byte == escape_byte, decode escaped byte
        byte_encoded = data[i + 1]
        byte_decoded = escape_byte ^ byte_encoded
        result += byte_decoded.to_bytes(1, 'big') # get the bytestring of this byte
        i += 2 # processed 2 byte for encoded byte
    # while loop done

    return result

# for uart protocal, -------------------- TB Tested --------------------------
def uart_encoder(data):
    # encode escape bytes
    result = b''
    for byte in data:
        # print("[R]", result)
        if byte < escape_byte:
            result += byte.to_bytes(1, 'big') # get the bytestring of this byte
            continue

        # byte >= escape_byte, encode escaped byte
        byte_encoded = escape_byte ^ byte
        result += escape_byte.to_bytes(1, 'big') # get the bytestring of this byte
        result += byte_encoded.to_bytes(1, 'big') # get the bytestring of this byte
    # while loop done

    return result


def sent_data(ser, data):
    ser.write(b'\xff')
    data_encoded = uart_encoder(data)
    ser.write(data_encoded)
    ser.write(b'\xfe')
    
    print("[Raw] >>", data)
    print("[END] >>", data_encoded)

def uart_read_message(serial_connection):
    # read the entire message base on our uart escape byte protocal
    # TB Finish, rn is the old 3 special end_of_message bytes
    data = b''
    while True:
        if serial_connection.in_waiting > 0:  # Check if there is data available to read
            byte = serial_connection.read() # Read and decode the data
            # print(byte, end="")
            if byte == b'\xff':
                # start of message
                data = b''
                continue
            if byte == b'\xfe':
                # found end of message sequence
                break
                
            data += byte

    # decode the raw uart signals
    print("[Enc] >>", data)
    return uart_decoder(data)

# Function to continuously read data from the serial port
def read_serial(ser):
    data = b''
    while True:
        if ser.in_waiting > 0:  # Check if there is data available to read
            data = uart_read_message(ser)
            print("[Dec] >>", data)

# Main function
def main():
    # Open the serial port
    ser = serial.Serial(port, baud_rate)  # Replace with your serial port and baud rate
    
    # Create and start the thread for reading from the serial port
    read_thread = threading.Thread(target=read_serial, args=(ser,), daemon=True)
    read_thread.start()
    
    while True:
        bytes_send = b'SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\xfb\xfc\xfd'
        sent_data(ser, bytes_send)
        # print('.', end = "")
        time.sleep(5)

if __name__ == "__main__":
    main()


[Raw] >> b'SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\xfb\xfc\xfd'
[END] >> b'SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\x00\xfa\x01\xfa\x06\xfa\x07'
[Enc] >> b'\x00\x11SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\x00\x00\xfa\x00\x01\xfa\x00\x06\xfa\x00\x07'
[Dec] >> b'\x00\x11SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\x00\xfa\x01\xfa\x06\xfa\x07'
[Enc] >> b'\x00\x11SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\x00\xfa\x01\xfa\x06\xfa\x07'
[Dec] >> b'\x00\x11SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\xfb\xfc\xfd'
[Enc] >> b'\x00\x11SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\x00\xfa\x01\xfa\x06\xfa\x07\xfa\x00\x06\xfa\x00\x07'
[Dec] >> b'\x00\x11SEND-\x00\x00![D]\x03GP

In [2]:
byte = b'\xa0'
print(byte[0])

160


In [3]:
byte = b'Z'
print(byte[0])

90


In [1]:
bytes = b'SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\xfb\xfc\xfd'
print(bytes)

b'SEND-\x00\x00![D]\x03GPS\x06\x04\x05\x04\x05\x04\x05LDC\x02\x88\xefIDX\x01\x8dESP\x04\xfa\xfb\xfc\xfd'
